In [1]:
is_startup = True

In [2]:
if is_startup:
  !pip install datasets

In [3]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/Detalytics/chatbot/"
sys.path.insert(0, project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
if is_startup:
  os.chdir(project_path)
  print (os.getcwd())
  !pip install -e .
  !pip install -e alutils

/content/drive/MyDrive/Detalytics/chatbot
Obtaining file:///content/drive/MyDrive/Detalytics/chatbot
  Preparing metadata (setup.py) ... done
  Attempting uninstall: chatbot
    Found existing installation: chatbot 0.0.0
    Uninstalling chatbot-0.0.0:
      Successfully uninstalled chatbot-0.0.0
  Running setup.py develop for chatbot
Obtaining file:///content/drive/MyDrive/Detalytics/chatbot/alutils
  Preparing metadata (setup.py) ... done
  Attempting uninstall: alutils
    Found existing installation: alutils 0.0.0
    Uninstalling alutils-0.0.0:
      Successfully uninstalled alutils-0.0.0
  Running setup.py develop for alutils


In [5]:
from datasets import load_dataset
from chatbot import constants
from alutils import alos

data_name = 'daily_dialog'
data_dir = alos.gen_dir(f"{constants.DATA_PATH}{data_name}")
dataset = load_dataset(data_name, cache_dir=data_dir)

for id, sample in enumerate(dataset['train']):
    if id > 10: break
    dialogs = sample['dialog']
    for dialog in dialogs:
        print (dialog)
        # print (dialog['text'])
    print ('------------------------------------------')


Say , Jim , how about going for a few beers after dinner ? 
 You know that is tempting but is really not good for our fitness . 
 What do you mean ? It will help us to relax . 
 Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? 
 I guess you are right.But what shall we do ? I don't feel like sitting at home . 
 I suggest a walk over to the gym where we can play singsong and meet some of our friends . 
 That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . 
 Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . 
 Good.Let ' s go now . 
 All right . 
------------------------------------------
Can you do push-ups ? 
 Of course I can . It's a piece of cake ! Believe it or not , I can do 30 push-ups a minute . 
 Really ? I think that's impossible ! 
 You mean 30 push-ups ? 
 Yeah ! 
 It's easy . If you do exerc

In [6]:
if is_startup:
  !pip install transformers[torch]
  !pip install accelerate -U

In [29]:
#train a chatbot: https://www.perplexity.ai/search/conda-how-to-DzRf10j2S_Wp9Cq5_Q5Vjw?s=c

from chatbot import constants
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

dataset_name = 'conv_ai_2'
llm_model = 'gpt2'

# Load the dataset
dataset = load_dataset(dataset_name)


# Preprocess the data
def preprocess_function(examples):
    dialog_text = []
    for id, dialog in enumerate(examples['dialog']):
        text = []
        for turn in dialog:
            # print (type(turn), turn)
            speaker_token = '[USER]' if turn['sender_class'] == 'Human' else '[BOT]'
            text.append(f"{speaker_token} {turn['text']}")
            # print (text[-1])
        dialog_text.append(' '.join(text))
        if id == 0:
            print (dialog_text[-1][:512])
    print (f'#dialog_text: {len(dialog_text)}')
    return tokenizer(dialog_text, truncation=True, padding='max_length', max_length=512)

# def preprocess_function(examples):
#     print (len(examples))
#     # Join the 'text' field from each dictionary into a single string
#     dialog_text = [' '.join(turn['text'] for turn in dialog) for dialog in examples['dialog']]
#     for id, dialog in enumerate(dialog_text[:5]):
#       print (id, dialog)
#       print ('-----------------------------/n/n')
#     return tokenizer(dialog_text, truncation=True, padding='max_length', max_length=128)

# Tokenize the dataset
tokenizer = GPT2Tokenizer.from_pretrained(llm_model)
# Set the padding token to the EOS token
tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3495 [00:00<?, ? examples/s]

[BOT] I love iphone! i just bought new iphone! [USER] Thats good for you, i'm not very into new tech [BOT] I am a college student and i am a college student [USER] I am go to gym and live on donations [BOT] I am a vegan and i am in the midwest [USER] So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat [BOT] I would not mind having them in the office that would be hard for me [USER] Dogs or vegan in office? [BOT] I am a vegetarian so i am vegan [USER] Strange answer
#dialog_text: 1000
[BOT] i m alcoholic   but i go to college during the week to become a lawyer   [USER] You are very young to be an alcoholic. [BOT] i m an alcoholic so it is very hard [USER] I understand. I am recovering alcoholic. [BOT] i do not drink alcohol at all   i ve do many alcoholics in my family [USER] Do you have mom and dad? [BOT] do you live with your mom and dad   [USER] No, I live along. I need free time to take my meditations. [BOT] that sounds like a really fun ti

In [20]:
# Prepare for training
model = GPT2LMHeadModel.from_pretrained(llm_model)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define the training arguments
training_args = TrainingArguments(
    output_dir=alos.gen_dir(f"{constants.CKPT_PATH}{llm_model}-{dataset_name}"),
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
)

# Create the Trainer and train
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)

trainer.train()

Step,Training Loss
500,2.322500
1000,1.902100
1500,1.888400
2000,1.794200
2500,1.765200
3000,1.731300
3500,1.599900
4000,1.476100
4500,1.443700
5000,1.422700


TrainOutput(global_step=17475, training_loss=1.3462247337155757, metrics={'train_runtime': 776.2043, 'train_samples_per_second': 22.513, 'train_steps_per_second': 22.513, 'total_flos': 1141519564800000.0, 'train_loss': 1.3462247337155757, 'epoch': 5.0})

In [23]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_response(prompt):
    # Prepend the user token to the prompt
    prompt = '[USER] ' + prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=128, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the bot token from the response
    # response = response.replace('[BOT] ', '')
    return response

# Test the chatbot with a sample input
user_input = "What is your favorite hobby?"
response = generate_response(user_input)
print("User input:", user_input)
print("Chatbot response:", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User input: What is your favorite hobby?
Chatbot response: [USER] What is your favorite hobby? [USER] What is your favorite meal? [USER] Do you like to dance? [USER] Do you like to dance? [USER] Do you like to dance? [USER] What is your favorite color? [USER] Bye [USER] Okay [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER] Bye [USER
